In [1]:
%matplotlib inline
from memory_profiler import memory_usage
import os
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import h5py

In [2]:
import keras
from keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D, GaussianNoise
from keras.models import Sequential
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,TensorBoard,ProgbarLogger
from keras import regularizers, optimizers
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import f1_score
import librosa
import librosa.display
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import random
import warnings
warnings.filterwarnings('ignore')

#object serialization
import _pickle as cPickle  #python 3 change
import os  

%matplotlib inline

Using TensorFlow backend.


In [45]:
#enable memory profiler for memory management usage %%memit 
%load_ext memory_profiler

#enable garbage collection control
import gc
gc.enable()

In [46]:
from tqdm import tqdm, tqdm_notebook

In [8]:
# when set to TRUE, training data get loaded from a saved serialized data object file 
# All audio files data get saved to a serialized object file to save reloading time on training runs 

# Note: 
# On first time run, if serialized file doesn't exist, this flag will get overrident 

SKIP_AUDIO_RELOAD = False

In [48]:
#location of the sound files
INPUT_PATH='/Users/Bruker/Documents/prosjekt/JS'

In [49]:
import wave
import contextlib
fname = '/Users/Bruker/Documents/prosjekt/JS/Lok1_20170607_000000.wav'
with contextlib.closing(wave.open(fname,'r')) as f:
    frames = f.getnframes()
    rate = f.getframerate()
    duration = frames / float(rate)
    print(duration)

67108.016


In [50]:
# From https://stackoverflow.com/questions/36799902/how-to-splice-an-audio-file-wav-format-into-1-sec-splices-in-python

from pydub.utils import make_chunks

myaudio = AudioSegment.from_file("/Users/Bruker/Documents/prosjekt/JS/Lok1_20170607_000000.wav" , "wav") 
chunk_length_ms = 4000 # pydub calculates in millisec
chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of one sec

#Export all of the individual chunks as wav files
for i, chunk in enumerate(chunks):
    chunk_name = "chunk{0}.wav".format(i)
    print ("exporting", chunk_name)
    chunk.export(chunk_name, format="wav")

exporting chunk0.wav
exporting chunk1.wav
exporting chunk2.wav
exporting chunk3.wav
exporting chunk4.wav
exporting chunk5.wav
exporting chunk6.wav
exporting chunk7.wav
exporting chunk8.wav
exporting chunk9.wav
exporting chunk10.wav
exporting chunk11.wav
exporting chunk12.wav
exporting chunk13.wav
exporting chunk14.wav
exporting chunk15.wav
exporting chunk16.wav
exporting chunk17.wav
exporting chunk18.wav
exporting chunk19.wav
exporting chunk20.wav
exporting chunk21.wav
exporting chunk22.wav
exporting chunk23.wav
exporting chunk24.wav
exporting chunk25.wav
exporting chunk26.wav
exporting chunk27.wav
exporting chunk28.wav
exporting chunk29.wav
exporting chunk30.wav
exporting chunk31.wav
exporting chunk32.wav
exporting chunk33.wav
exporting chunk34.wav
exporting chunk35.wav
exporting chunk36.wav
exporting chunk37.wav
exporting chunk38.wav
exporting chunk39.wav
exporting chunk40.wav
exporting chunk41.wav
exporting chunk42.wav
exporting chunk43.wav
exporting chunk44.wav
exporting chunk45.wa

exporting chunk361.wav
exporting chunk362.wav
exporting chunk363.wav
exporting chunk364.wav
exporting chunk365.wav
exporting chunk366.wav
exporting chunk367.wav
exporting chunk368.wav
exporting chunk369.wav
exporting chunk370.wav
exporting chunk371.wav
exporting chunk372.wav
exporting chunk373.wav
exporting chunk374.wav
exporting chunk375.wav
exporting chunk376.wav
exporting chunk377.wav
exporting chunk378.wav
exporting chunk379.wav
exporting chunk380.wav
exporting chunk381.wav
exporting chunk382.wav
exporting chunk383.wav
exporting chunk384.wav
exporting chunk385.wav
exporting chunk386.wav
exporting chunk387.wav
exporting chunk388.wav
exporting chunk389.wav
exporting chunk390.wav
exporting chunk391.wav
exporting chunk392.wav
exporting chunk393.wav
exporting chunk394.wav
exporting chunk395.wav
exporting chunk396.wav
exporting chunk397.wav
exporting chunk398.wav
exporting chunk399.wav
exporting chunk400.wav
exporting chunk401.wav
exporting chunk402.wav
exporting chunk403.wav
exporting c

exporting chunk1070.wav
exporting chunk1071.wav
exporting chunk1072.wav
exporting chunk1073.wav
exporting chunk1074.wav
exporting chunk1075.wav
exporting chunk1076.wav
exporting chunk1077.wav
exporting chunk1078.wav
exporting chunk1079.wav
exporting chunk1080.wav
exporting chunk1081.wav
exporting chunk1082.wav
exporting chunk1083.wav
exporting chunk1084.wav
exporting chunk1085.wav
exporting chunk1086.wav
exporting chunk1087.wav
exporting chunk1088.wav
exporting chunk1089.wav
exporting chunk1090.wav
exporting chunk1091.wav
exporting chunk1092.wav
exporting chunk1093.wav
exporting chunk1094.wav
exporting chunk1095.wav
exporting chunk1096.wav
exporting chunk1097.wav
exporting chunk1098.wav
exporting chunk1099.wav
exporting chunk1100.wav
exporting chunk1101.wav
exporting chunk1102.wav
exporting chunk1103.wav
exporting chunk1104.wav
exporting chunk1105.wav
exporting chunk1106.wav
exporting chunk1107.wav
exporting chunk1108.wav
exporting chunk1109.wav
exporting chunk1110.wav
exporting chunk1

exporting chunk1752.wav
exporting chunk1753.wav
exporting chunk1754.wav
exporting chunk1755.wav
exporting chunk1756.wav
exporting chunk1757.wav
exporting chunk1758.wav
exporting chunk1759.wav
exporting chunk1760.wav
exporting chunk1761.wav
exporting chunk1762.wav
exporting chunk1763.wav
exporting chunk1764.wav
exporting chunk1765.wav
exporting chunk1766.wav
exporting chunk1767.wav
exporting chunk1768.wav
exporting chunk1769.wav
exporting chunk1770.wav
exporting chunk1771.wav
exporting chunk1772.wav
exporting chunk1773.wav
exporting chunk1774.wav
exporting chunk1775.wav
exporting chunk1776.wav
exporting chunk1777.wav
exporting chunk1778.wav
exporting chunk1779.wav
exporting chunk1780.wav
exporting chunk1781.wav
exporting chunk1782.wav
exporting chunk1783.wav
exporting chunk1784.wav
exporting chunk1785.wav
exporting chunk1786.wav
exporting chunk1787.wav
exporting chunk1788.wav
exporting chunk1789.wav
exporting chunk1790.wav
exporting chunk1791.wav
exporting chunk1792.wav
exporting chunk1

exporting chunk2434.wav
exporting chunk2435.wav
exporting chunk2436.wav
exporting chunk2437.wav
exporting chunk2438.wav
exporting chunk2439.wav
exporting chunk2440.wav
exporting chunk2441.wav
exporting chunk2442.wav
exporting chunk2443.wav
exporting chunk2444.wav
exporting chunk2445.wav
exporting chunk2446.wav
exporting chunk2447.wav
exporting chunk2448.wav
exporting chunk2449.wav
exporting chunk2450.wav
exporting chunk2451.wav
exporting chunk2452.wav
exporting chunk2453.wav
exporting chunk2454.wav
exporting chunk2455.wav
exporting chunk2456.wav
exporting chunk2457.wav
exporting chunk2458.wav
exporting chunk2459.wav
exporting chunk2460.wav
exporting chunk2461.wav
exporting chunk2462.wav
exporting chunk2463.wav
exporting chunk2464.wav
exporting chunk2465.wav
exporting chunk2466.wav
exporting chunk2467.wav
exporting chunk2468.wav
exporting chunk2469.wav
exporting chunk2470.wav
exporting chunk2471.wav
exporting chunk2472.wav
exporting chunk2473.wav
exporting chunk2474.wav
exporting chunk2

...

exporting chunk7890.wav
exporting chunk7891.wav
exporting chunk7892.wav
exporting chunk7893.wav
exporting chunk7894.wav
exporting chunk7895.wav
exporting chunk7896.wav
exporting chunk7897.wav
exporting chunk7898.wav
exporting chunk7899.wav
exporting chunk7900.wav
exporting chunk7901.wav
exporting chunk7902.wav
exporting chunk7903.wav
exporting chunk7904.wav
exporting chunk7905.wav
exporting chunk7906.wav
exporting chunk7907.wav
exporting chunk7908.wav
exporting chunk7909.wav
exporting chunk7910.wav
exporting chunk7911.wav
exporting chunk7912.wav
exporting chunk7913.wav
exporting chunk7914.wav
exporting chunk7915.wav
exporting chunk7916.wav
exporting chunk7917.wav
exporting chunk7918.wav
exporting chunk7919.wav
exporting chunk7920.wav
exporting chunk7921.wav
exporting chunk7922.wav
exporting chunk7923.wav
exporting chunk7924.wav
exporting chunk7925.wav
exporting chunk7926.wav
exporting chunk7927.wav
exporting chunk7928.wav
exporting chunk7929.wav
exporting chunk7930.wav
exporting chunk7

exporting chunk8572.wav
exporting chunk8573.wav
exporting chunk8574.wav
exporting chunk8575.wav
exporting chunk8576.wav
exporting chunk8577.wav
exporting chunk8578.wav
exporting chunk8579.wav
exporting chunk8580.wav
exporting chunk8581.wav
exporting chunk8582.wav
exporting chunk8583.wav
exporting chunk8584.wav
exporting chunk8585.wav
exporting chunk8586.wav
exporting chunk8587.wav
exporting chunk8588.wav
exporting chunk8589.wav
exporting chunk8590.wav
exporting chunk8591.wav
exporting chunk8592.wav
exporting chunk8593.wav
exporting chunk8594.wav
exporting chunk8595.wav
exporting chunk8596.wav
exporting chunk8597.wav
exporting chunk8598.wav
exporting chunk8599.wav
exporting chunk8600.wav
exporting chunk8601.wav
exporting chunk8602.wav
exporting chunk8603.wav
exporting chunk8604.wav
exporting chunk8605.wav
exporting chunk8606.wav
exporting chunk8607.wav
exporting chunk8608.wav
exporting chunk8609.wav
exporting chunk8610.wav
exporting chunk8611.wav
exporting chunk8612.wav
exporting chunk8

exporting chunk9254.wav
exporting chunk9255.wav
exporting chunk9256.wav
exporting chunk9257.wav
exporting chunk9258.wav
exporting chunk9259.wav
exporting chunk9260.wav
exporting chunk9261.wav
exporting chunk9262.wav
exporting chunk9263.wav
exporting chunk9264.wav
exporting chunk9265.wav
exporting chunk9266.wav
exporting chunk9267.wav
exporting chunk9268.wav
exporting chunk9269.wav
exporting chunk9270.wav
exporting chunk9271.wav
exporting chunk9272.wav
exporting chunk9273.wav
exporting chunk9274.wav
exporting chunk9275.wav
exporting chunk9276.wav
exporting chunk9277.wav
exporting chunk9278.wav
exporting chunk9279.wav
exporting chunk9280.wav
exporting chunk9281.wav
exporting chunk9282.wav
exporting chunk9283.wav
exporting chunk9284.wav
exporting chunk9285.wav
exporting chunk9286.wav
exporting chunk9287.wav
exporting chunk9288.wav
exporting chunk9289.wav
exporting chunk9290.wav
exporting chunk9291.wav
exporting chunk9292.wav
exporting chunk9293.wav
exporting chunk9294.wav
exporting chunk9

exporting chunk9936.wav
exporting chunk9937.wav
exporting chunk9938.wav
exporting chunk9939.wav
exporting chunk9940.wav
exporting chunk9941.wav
exporting chunk9942.wav
exporting chunk9943.wav
exporting chunk9944.wav
exporting chunk9945.wav
exporting chunk9946.wav
exporting chunk9947.wav
exporting chunk9948.wav
exporting chunk9949.wav
exporting chunk9950.wav
exporting chunk9951.wav
exporting chunk9952.wav
exporting chunk9953.wav
exporting chunk9954.wav
exporting chunk9955.wav
exporting chunk9956.wav
exporting chunk9957.wav
exporting chunk9958.wav
exporting chunk9959.wav
exporting chunk9960.wav
exporting chunk9961.wav
exporting chunk9962.wav
exporting chunk9963.wav
exporting chunk9964.wav
exporting chunk9965.wav
exporting chunk9966.wav
exporting chunk9967.wav
exporting chunk9968.wav
exporting chunk9969.wav
exporting chunk9970.wav
exporting chunk9971.wav
exporting chunk9972.wav
exporting chunk9973.wav
exporting chunk9974.wav
exporting chunk9975.wav
exporting chunk9976.wav
exporting chunk9

KeyboardInterrupt: 

In [63]:
INPUT_PATH='/Users/Bruker/prosjektH19'

TEST_INPUT=INPUT_PATH+'/test2'
TEST_AUDIO_DIR=TEST_INPUT+'/Test2'

In [64]:
def load_input_data(pd, filepath):
    # Read Data
    data = pd.read_csv(filepath)
    return data

In [67]:
# test file
TEST_FILE=TEST_INPUT+'/Chunk.csv'

#show info
test_input=load_input_data(pd,TEST_FILE)
test_input.head()

,file_ID
0,chunk0
1,chunk1
2,chunk2
3,chunk3
4,chunk4


In [68]:
# test data size
valid_test_data = test_input[['file_ID']] 
valid_test_data.count()


file_ID    16779
dtype: int64

In [70]:
# sample-1 load
sample1=TEST_AUDIO_DIR+'/Chunk1.wav'
duration=3.0
sr=22050

In [74]:
#test audio files
valid_test_data['path'] = TEST_AUDIO_DIR+'/' + test_input['file_ID'].astype('str') +".wav"
print ("sample",valid_test_data.path[1])
valid_test_data.head(5)

sample /Users/Bruker/prosjektH19/test2/Test2/chunk1.wav


,file_ID,path
0,chunk0,/Users/Bruker/prosjektH19/test2/Test2/chunk0.wav
1,chunk1,/Users/Bruker/prosjektH19/test2/Test2/chunk1.wav
2,chunk2,/Users/Bruker/prosjektH19/test2/Test2/chunk2.wav
3,chunk3,/Users/Bruker/prosjektH19/test2/Test2/chunk3.wav
4,chunk4,/Users/Bruker/prosjektH19/test2/Test2/chunk4.wav


In [75]:
#
# set duration on audio loading to make audio content to ensure each training data have same size
# 
#fix the load audio file size
audio_play_duration=4

def load_audio_file(file_path, duration=4, sr=22050):
    #load 4 seconds audio file, sr=22050
    # To preserve the native sampling rate of the file, use sr=None
    input_length=sr*duration
    # Load an audio file as a floating point time series.
    # y : np.ndarray [shape=(n,) or (2, n)] - audio time series
    # sr : number > 0 [scalar] - sampling rate of y
    y, sr = librosa.load(file_path,sr=sr, duration=duration)
    dur = librosa.get_duration(y=y)
    #pad output if audio file less than duration
    # Use edge-padding instead of zeros
    #librosa.util.fix_length(y, 10, mode='edge')
    if (round(dur) < duration):
        offset = len(y) - round(input_length)
        print ("fixing audio length :", file_path)
        print ("input:", round(input_length), " load:", len(y) , " offset:", offset)
        y = librosa.util.fix_length(y, round(input_length))      
        y = librosa.util.normalize(y)
    # using a pre-computed power spectrogram
    # Short-time Fourier transform (STFT)
    #D = np.abs(librosa.stft(y))**2
    #ps = librosa.feature.melspectrogram(S=D)   
    ps = librosa.feature.melspectrogram(y=y, sr=2000)
    return ps

In [76]:

%%time
#%%memit 
# Dataset
test_audio_data = [] 
test_object_file='saved_train_audio_data.p'

#override the reload flag if serized file doesn't exist
if not os.path.isfile(test_object_file):
    SKIP_AUDIO_RELOAD = False

#load training data
if SKIP_AUDIO_RELOAD is True:
    print ("skip re-loading TEST data from audio files")
else:
    print ("loading test audio data. please wait!")
    for row in tqdm(valid_test_data.itertuples()):
            ps = load_audio_file(file_path=row.path, duration=4.0)
            if ps.shape != (128, 130): continue
            test_audio_data.append( (ps) ) 
    print("Number of test samples: ", len(test_audio_data))
# this step took sometime to finish, 16 778 files
# total: 35min 53s

loading test audio data. please wait!


16777it [35:53,  7.72it/s]

fixing audio length : /Users/Bruker/prosjektH19/test2/Test2/chunk16777.wav
input: 66150  load: 353  offset: -65797


FileNotFoundError: [Errno 2] No such file or directory: '/Users/Bruker/prosjektH19/test2/Test2/chunk16778.wav'

In [77]:
# save as numpy array to not have to load all the data for each run
np.save('chunk.npy', test_audio_data)

In [9]:
# load saved data
test_audio_data = []
test_object_file='saved_test_audio_data.p'
if SKIP_AUDIO_RELOAD is True:
    test_audio_data = cPickle.load(open(test_object_file, 'rb'))
    print ("loaded test data [%s] records from object file" % len(test_audio_data))      
else:
    cPickle.dump(test_audio_data, open(test_object_file, 'wb')) 
    print ("save loaded test data :", len(test_audio_data))

save loaded test data : 0


In [12]:
test_audio_data = np.load('chunk.npy')
dataset = test_audio_data

In [13]:
random.shuffle(dataset)

In [14]:
X_test = zip(*dataset)

In [16]:
from keras.models import load_model

model = load_model('best_model_LSTM.h5')

In [32]:
import time

t = time.time()
y_pred = model.predict(dataset)
print((time.time()-t),'Process time in seconds')

43.284690856933594 Process time in seconds
